In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Polygon, LineString, Point
import statsmodels.formula.api as smf #회귀식 도출 위한 모듈

import openpyxl

# user-defined modules
import _path

In [2]:
path  = _path.path
pathX = _path.pathX
pathY = _path.pathX

# 실거래 이력이 있는 주소지 - 1) 배정학교 6학년 비율 점수 2) 반경 500m 내 학원 수

## 1. 실거래 이력이 있는 주소지 - 초등학교 배정

In [5]:
V_LOCATION = pd.read_excel(path+"\\"+"V_LOCATION.xlsx",sheet_name = 0, header=0)
SEOUL_ELSCH_INFO = pd.read_excel(path+"\\"+"SEOUL_ELSCH_INFO.xlsx",sheet_name = 0, header=0)

In [6]:
ELSCHOOL = SEOUL_ELSCH_INFO[['SCH_NAME', 'HAKGUDO_NM', 'ADDR', 'GRADE1', 'GRADE6', 'TOT_NUM', 'GRADE1_PROP', 'GRADE6_PROP', 'SCORE_GRADE6_PROP', 'LAT', 'LNG']]
ELSCHOOL.columns = ['sch_name', 'hakgudo_nm', 'addr', 'grade1', 'grade6', 'tot_num', 'grade1_prop', 'grade6_prop', 'score_grade6_prop', 'lat', 'lng']

In [7]:
ELSCHOOL['geometry'] = ELSCHOOL.apply(lambda row : Point([row['lng'], row['lat']]), axis=1)
# dataframe -> geodataframe 변환
ELSCHOOL = gpd.GeoDataFrame(ELSCHOOL, geometry='geometry')
print(ELSCHOOL.crs)
# 기본 위경도좌표는 EPSG4326
ELSCHOOL.crs = {'init': 'epsg:4326'}
ELSCHOOL = ELSCHOOL.to_crs({'init':'epsg:5186'})
print(ELSCHOOL.crs)

None
+init=epsg:5186 +type=crs


<ipython-input-7-809056ac7e0f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ELSCHOOL['geometry'] = ELSCHOOL.apply(lambda row : Point([row['lng'], row['lat']]), axis=1)
C:\ProgramData\Anaconda3\lib\site-packages\pyproj\crs\crs.py:68: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [ ]:
# 1-1) 학구도 확인해서 초등학교 배정
for idx_t, trans in V_LOCATION.iterrows():
    for idx_s, school in ELSCHOOL.iterrows():
        if(trans.geometry.within(school.hakgudo_geo)==True):
            V_LOCATION.loc[idx_t,'sch_name'] = school.sch_name
            break
        else:
            continue

In [ ]:
# 초등학교 배정안된 주소 확인: 1989개
V_LOCATION[V_LOCATION.sch_name.isnull()]

In [ ]:
# V_LOCATION.sch_name = Nan -> 빈칸으로 만들기
V_LOCATION = V_LOCATION.replace(np.nan, '', regex=True)

In [ ]:
# 1-2) 학구도로 배정안된 주소 가장 가까운 학교 배정하기
def min_dist(point, gpd2):
    gpd2['Dist'] = gpd2.apply(lambda row: point.distance(row.geometry),axis=1)
    geoseries = gpd2.iloc[gpd2['Dist'].argmin()]
    return geoseries.sch_name

In [ ]:
for idx_t, trans in V_LOCATION.iterrows():
    if (trans.sch_name == '') :
        V_LOCATION.loc[idx_t,'sch_name'] = min_dist(trans.geometry, ELSCHOOL)
    else: 
        continue

## 2. 실거래 이력이 있는 주소지 - 반경 500m 내 학원 수(edu_hakwon_cnt.ipynb)

# 실거래 이력이 있는 주소지 - 1) 배정학교 6학년 비율 점수 2) 반경 500m 내 학원 수 (엑셀파일 load ver.)

In [12]:
filename = "V_LOCATION_edited_3.xlsx"
V_LOCATION = pd.read_excel(path+"\\"+filename,sheet_name = 0, header=0)

In [19]:
V_LOCATION['hakwon_cnt_log'] = V_LOCATION['hakwon_cnt'].apply(lambda x: np.log(x+1))

# 거래 이력에 교육여건 관련 데이터 붙이기

In [3]:
# 실거래 이력 load
TRANSACTION_VILLA = pd.read_excel(path+"\\"+"TRANSACTION_VILLA.xlsx",sheet_name = 0, header=0)

In [4]:
TRANSACTION = TRANSACTION_VILLA[['addr', 'age','floor', 'cost','cost_reflected','cost_reflected_log']]

In [21]:
# 거래 이력에 배정 초등학교 정보 add
TRANSACTION = pd.merge(TRANSACTION, V_LOCATION[['addr','lat','lng','geometry','sch_name','score_grade6_prop','hakwon_cnt','hakwon_cnt_log']], on='addr', how='left')

In [22]:
V_LOCATION.shape

(78040, 11)

In [23]:
V_LOCATION.describe()

,lat,lng,score_grade6_prop,hakwon_cnt,EMD_CD,hakwon_cnt_log
count,78040.000000,78040.000000,78040.000000,78040.000000,7.804000e+04,78040.000000
mean,37.554410,126.974284,1.823665,47.890928,1.144061e+07,3.607742
std,0.056213,0.090120,3.634215,48.799482,1.660529e+05,0.796583
min,35.335007,126.798744,-15.526000,0.000000,1.111010e+07,0.000000
25%,37.505626,126.909950,-0.350000,24.000000,1.130510e+07,3.218876
50%,37.549965,126.962194,1.811000,39.000000,1.144012e+07,3.688879
75%,37.599170,127.042539,3.890000,61.000000,1.159010e+07,4.127134
max,37.687177,129.037169,17.435000,1004.000000,1.174011e+07,6.912743


In [24]:
V_LOCATION.head()

,addr,gu,dong,lat,lng,geometry,sch_name,score_grade6_prop,hakwon_cnt,EMD_CD,hakwon_cnt_log
0,서울특별시 중구 광희동2가 장충단로8가길 18-2,중구,광희동2가,37.562857,127.007960,POINT (200703.2937658533 551480.5026546286),서울장충초등학교,6.865,3,11140146,1.386294
1,서울특별시 중구 광희동2가 장충단로8나길 11-28,중구,광희동2가,37.563070,127.008599,POINT (200759.7480674413 551504.1037245411),서울장충초등학교,6.865,3,11140146,1.386294
2,서울특별시 중구 남산동2가 퇴계로20나길 45,중구,남산동2가,37.557155,126.985474,POINT (198716.5381771795 550847.7286095693),서울남산초등학교,-9.551,4,11140129,1.609438
3,서울특별시 중구 남창동 퇴계로2길 23,중구,남창동,37.556544,126.977476,POINT (198009.8257770552 550780.0097542248),서울남산초등학교,-9.551,2,11140112,1.098612
4,서울특별시 중구 남창동 퇴계로6가길 30-7,중구,남창동,37.556540,126.977835,POINT (198041.5376161356 550779.5471647036),서울남산초등학교,-9.551,1,11140112,0.693147


In [25]:
TRANSACTION

,addr,age,floor,cost,cost_reflected,cost_reflected_log,lat,lng,geometry,sch_name,score_grade6_prop,hakwon_cnt,hakwon_cnt_log
0,서울특별시 강서구 화곡동 초록마을로24길 27-11,9.0,1,157500000,149739975,18.824411,37.539104,126.850152,POINT (186756.5773103896 548854.7601730863),서울화일초등학교,-1.399,26.0,3.295837
1,서울특별시 강서구 화곡동 초록마을로24길 21-9,8.0,-1,160000000,150236800,18.827723,37.538891,126.850495,POINT (186786.8273415241 548831.0715477165),서울화일초등학교,-1.399,22.0,3.135494
2,서울특별시 강서구 화곡동 초록마을로24길 21,8.0,4,150000000,140847000,18.763185,37.539032,126.850837,POINT (186817.0602029711 548846.595173208),서울화일초등학교,-1.399,20.0,3.044522
3,서울특별시 강서구 화곡동 강서로18라길 6,24.0,2,121000000,113616579,18.548340,37.534759,126.848531,POINT (186612.5021680367 548372.7143265564),서울신정초등학교,1.429,49.0,3.912023
4,서울특별시 강서구 화곡동 까치산로18길 44-13,9.0,1,124000000,116433519,18.572831,37.545873,126.850574,POINT (186794.9877836591 549605.9782850359),서울화곡초등학교,2.716,28.0,3.367296
...,...,...,...,...,...,...,...,...,...,...,...,...,...
467593,서울특별시 종로구 누상동 필운대로5가길 43,19.0,4,520000000,547253200,20.120422,37.580451,126.966941,POINT (197079.8558931934 553433.716339696),서울청운초등학교,3.505,22.0,3.135494
467594,서울특별시 종로구 누상동 옥인길 59-12,19.0,3,305000000,320985050,19.586905,37.581046,126.965595,POINT (196961.0490559293 553499.7639050998),서울청운초등학교,3.505,18.0,2.944439
467595,서울특별시 종로구 누상동 옥인6길 37-8,19.0,4,315000000,331509150,19.619166,37.580357,126.965185,POINT (196924.7525303085 553423.3727399055),서울청운초등학교,3.505,11.0,2.484907
467596,서울특별시 종로구 누상동 옥인6길 37-8,20.0,4,340000000,362113260,19.707468,37.580357,126.965185,POINT (196924.7525303085 553423.3727399055),서울청운초등학교,3.505,11.0,2.484907


In [28]:
lm = smf.ols(formula = "cost_reflected_log ~ score_grade6_prop", data=TRANSACTION).fit()
print(lm.summary())

                            OLS Regression Results                            
Dep. Variable:     cost_reflected_log   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     12.11
Date:                Tue, 14 Sep 2021   Prob (F-statistic):           0.000502
Time:                        22:27:01   Log-Likelihood:            -3.5973e+05
No. Observations:              467552   AIC:                         7.195e+05
Df Residuals:                  467550   BIC:                         7.195e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept            19.1809      0.00

In [35]:
print("R-squared: ", lm.rsquared)
print("coefficient of score_grade6_prop: ",lm.params[1])
print("p-value of score_grade6_prop: ", lm.pvalues[1])


R-squared:  2.58930927637957e-05
coefficient of score_grade6_prop:  0.0007658127037924445
p-value of score_grade6_prop:  0.0005024745645543591


In [36]:
lm = smf.ols(formula = "cost_reflected_log ~ hakwon_cnt_log", data=TRANSACTION).fit()
print(lm.summary())

                            OLS Regression Results                            
Dep. Variable:     cost_reflected_log   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     1147.
Date:                Wed, 15 Sep 2021   Prob (F-statistic):          5.10e-251
Time:                        00:52:42   Log-Likelihood:            -3.5916e+05
No. Observations:              467552   AIC:                         7.183e+05
Df Residuals:                  467550   BIC:                         7.183e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         19.0632      0.004   5295.

In [37]:
print("R-squared: ", lm.rsquared)
print("coefficient of log(hakwon_cnt): ",lm.params[1])
print("p-value of log(hakwon_cnt): ", lm.pvalues[1])


R-squared:  0.002446250682426876
coefficient of log(hakwon_cnt):  0.03280148840557007
p-value of log(hakwon_cnt):  5.095191311071074e-251


# 변수 간 상관분석

In [48]:
V_LOCATION[['score_grade6_prop','hakwon_cnt_log']].corr()

,score_grade6_prop,hakwon_cnt_log
score_grade6_prop,1.000000,-0.001795
hakwon_cnt_log,-0.001795,1.000000


# 교육여건 주성분분석

In [38]:
from sklearn.preprocessing import StandardScaler 
from sklearn.decomposition import PCA


In [46]:
x = V_LOCATION[['score_grade6_prop','hakwon_cnt_log']]
#x = StandardScaler().fit_transform(x) # x 표준화

In [47]:
pca = PCA(n_components=1) # 주성분을 몇개로 할지 결정
printcipalComponents = pca.fit_transform(x)

pca.explained_variance_ratio_
#sum(pca.explained_variance_ratio_)

array([0.95415842])

In [ ]:
principalDf = pd.DataFrame(data=printcipalComponents, columns = ['principal component1', 'principal component2']) # 주성분으로 이루어진 df

V_LOCATION = pd.concat([V_LOCATION, principalDf], axis=1)



md = smf.ols(formula = "cost_reflected_log ~ principal_component1+principal_component2", data=TRANSACTION_VILLA_A)
result = md.fit()
print(result.summary())